# Sean's adaptation
#### Goal:
Use `JsonOutputParser` to get more stable output format when generate multi-queiries

In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

import rich

In [2]:
from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

from langchain_community.vectorstores import Chroma

/Users/sean_liu/miniconda3/envs/langchain/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/sean_liu/miniconda3/envs/langchain/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = DirectoryLoader('../../pdf_files/',glob="*.pdf",loader_cls=PyPDFLoader)
documents = loader.load()

# Split text into chunks

text_splitter  = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
text_chunks = text_splitter.split_documents(documents)

vectorstore = Chroma.from_documents(documents=text_chunks, 
                                    embedding=embedding,
                                    persist_directory="data/vectorstore")
vectorstore.persist()

retriever = vectorstore.as_retriever()

/var/folders/ns/fxpzwr797mz7hy09lrlmt2500000gn/T/ipykernel_82703/313147623.py:17: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [7]:
# from langchain import hub
# prompt = hub.pull("langchain-ai/rag-fusion-query-generation")

In [4]:
from pydantic import BaseModel, Field
from typing import List

class Multi_Queries(BaseModel):
    multi_queries: List[str]=Field(description="The new queries that rephrase user's query with different perspectives.")

In [5]:
from langchain_core.output_parsers import JsonOutputParser

multi_queries_parser = JsonOutputParser(pydantic_object=Multi_Queries)
multi_queries_format = multi_queries_parser.get_format_instructions()

In [7]:
from langchain_core.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate)

system_prompt = """You are a helpful assistant that generates multiple search queries based on a single input query.
Generate 4 queries.

{format_instructions}
"""

system_message = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        template=system_prompt,
        # input_variables=['format_instructions']
        partial_variables={'format_instructions': multi_queries_format}
    )
)
human_message = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template="Generate multiple search queries related to: {original_query}",
        input_variables=['original_query']
    )
)

prompt = ChatPromptTemplate.from_messages(
    [
        system_message,
        human_message
    ]
)

In [8]:
from langchain_core.runnables import RunnablePassthrough

generate_multi_queries =(
{"original_query": RunnablePassthrough()}
| prompt
| ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
| multi_queries_parser
| (lambda x: x['multi_queries'])
)

In [9]:
generate_multi_queries.invoke({"original_query": "What need to consider when using LLM to eval LLM generation?"})

['What factors should be considered when evaluating LLM outputs?',
 'How to assess LLM generation quality when using LLMs?',
 'What are the key considerations for using LLMs to evaluate their own outputs?',
 'What should I keep in mind when using an LLM for evaluating generated text?']

In [10]:
from langchain.load import dumps, loads

def rrf(results: list[list], k=60):
    fused_scores = {}
    for docs in results:
        # assumes the docs are returned in the order of relevance
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            fused_scores[doc_str] += 1/(rank+k)

    reranked_results = [
        (loads(doc_str), score) for doc_str, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    
    return reranked_results

In [11]:
rrf_chain = generate_multi_queries | retriever.map() | rrf

In [12]:
input = {"original_query": "What need to consider when using LLM to eval LLM generation?"}
final_result = rrf_chain.invoke(input)

/var/folders/ns/fxpzwr797mz7hy09lrlmt2500000gn/T/ipykernel_82703/3786747218.py:14: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  (loads(doc_str), score) for doc_str, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)


In [13]:
rich.print(final_result)

[
    (
        Document(
            metadata={
                'page': 0,
                'source': '../../pdf_files/Large Language Models are not Fair Evaluators(2305.17926v2).pdf'
            },
            page_content='ample, The evaluation pipeline of Vicuna (Zheng\net al., 2023) has gained significant 
interest and wide\nusage due to its simplicity and interpretability. It\nprompts GPT-4 to score and compare 
candidate\nresponses and provide explanations, making it a\nvaluable tool for evaluation. However, it is un-\nclear
how reliable LLMs are as evaluators, as they\nare known to be sensitive to textual instructions\nand inputs (Dong 
et al., 2022; Turpin et al., 2023;\narXiv:2305.17926v2  [cs.CL]  30 Aug 2023'
        ),
        0.06639344262295081
    ),
    (
        Document(
            metadata={
                'page': 8,
                'source': '../../pdf_files/Large Language Models are not Fair Evaluators(2305.17926v2).pdf'
            },
            page_content='is not well explored. In this paper, we critically\nexamine the LLMs-as-evaluator 
paradigm and un-\ncover a significant positional bias. Furthermore,\nwe propose three simple yet effective methods 
to\ncalibrate the positional bias to achieve reliable and\nfair evaluation results.\n7 Conclusion\nIn this paper, 
we reveal a systematic positional\nbias in evaluation with advanced ChatGPT/GPT-4\nmodels: by manipulating the 
order of candidate\nresponses during evaluation, the quality ranking'
        ),
        0.06401249024199844
    ),
    (
        Document(
            metadata={
                'page': 2,
                'source': '../../pdf_files/Large Language Models are not Fair Evaluators(2305.17926v2).pdf'
            },
            page_content='evaluators to analyze the characteristics of posi-\ntional bias in LLM evaluators. We 
find that:\nLLMs are sensitive to the position of responses.\nAs shown in Table 2, in the evaluation of 
“Vicuna-\n13B v.s. ChatGPT” and “Vicuna-13B v.s. Alpaca-\n13B”, when the order was changed, LLMs provide\ndifferent
evaluation results, e.g., the win rate of\nVicuna-13B extremely differs when Vicuna-13B is\nevaluated as Assistant 
1 and Assistant 2.\nTo empirically evaluate the sensitivity, we in-'
        ),
        0.032266458495966696
    ),
    (
        Document(
            metadata={
                'page': 7,
                'source': '../../pdf_files/Large Language Models are not Fair Evaluators(2305.17926v2).pdf'
            },
            page_content='and reduce conflict, showcasing how calibration\nenhances LLM robustness.\n5.4 
Fine-Grained Analysis of Evaluation\nQuality\nIn order to further analyze the evaluation capabili-\nties of the 
model, we perform a fine-grained anal-\nysis of the questions by dividing them into 9 cate-\ngories following Zheng
et al. (2023). We calculate\nthe performance of different evaluators within these\ncategories. As shown in Figure 
6, we find that: 1)\nIn certain complex tasks such as common-sense,'
        ),
        0.03225806451612903
    ),
    (
        Document(
            metadata={
                'page': 7,
                'source': '../../pdf_files/Large Language Models are not Fair Evaluators(2305.17926v2).pdf'
            },
            page_content='2020a,b), question answering (Min et al., 2019),\nROC story cloze (Cai, Tu, and Gimpel, 
2017;\nSchwartz et al., 2017), lexical inference (Levy et al.,\n2015), visual question answering (Goyal et 
al.,\n2017), information extraction (Wang et al., 2021,\n2022; Song et al., 2023a; Xia et al., 2023) and so 
on.\nLLMs are pre-trained using a vast amount of data\nfrom the internet, making it highly likely for them\nto 
learn biases present in those materials. Although'
        ),
        0.016666666666666666
    ),
    (
        Document(
            metadata={
                'page': 7,
                'source': '../../pdf_files/Large Language Models are not Fair Evaluators(2305.17926v2).pdf'
            }